# 패널에 대한 태양광 입사각

In [2]:
import pandas as pd
import numpy as np
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict
import re

In [61]:
key = 'your key'
# key = input('your api key >> ')

In [76]:
# url에서 응답 불러오기

def get_sun_json(region, date, key):
    '''
    region: ulsan or dangjin
    date: 년월일, ex-'20210606'
    key: your api key
    '''

    lat_long = {'ulsan': (35.4776509, 129.380778), 'dangjin': (37.0507527, 126.5102993)}
    
    url_base = 'http://apis.data.go.kr/B090041/openapi/service/SrAltudeInfoService/getLCSrAltudeInfo'
    queryParams = urlencode({
        quote_plus('locdate'): date,                    # 날짜
        quote_plus('latitude'): lat_long[region][0],    # 위도
        quote_plus('longitude'): lat_long[region][1],   # 경도
        quote_plus('dnYn'): 'y',                        # 위도경도 숫자 형식 체크
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')
    
    results_json = xmltodict.parse(results)
    data = json.loads(json.dumps(results_json))
    
    return data


def angle_change(angle_str):
    
    angle_num = re.findall('\d+', angle_str)
    
    angle = int(angle_num[0]) + (1/60) * int(angle_num[1])
    
    return angle


def get_incidentangle(region, date, key):
    '''
    해당 날짜의 0~23시의 태양광 입사각을 계산
    - 발전소의 지역마다 패널의 경사각, 방위각이 다르므로 지역마다 구분한다.
        - 울산 : 20도, 20도
        - 당진 : 30도, 30도
    
    태양광 입사각 계산은 아래 사이트를 참고
    https://www.sciencedirect.com/topics/engineering/solar-incidence-angle
    '''
    data_json = get_sun_json(region, date, key)
    data = data_json['response']['body']['items']['item']
    
    # 패널 설치 정보
    panel_region = {'ulsan': 20, 'dangjin': 30}
    lat_region = {'ulsan': 35.4776509, 'dangjin': 37.0507527}
        
    sin = np.sin
    cos = np.cos
    arccos = np.arccos
    pi = np.pi

    panel_tilt = panel_region[region] * pi / 180  # 패널 설치각(rad)
    panel_azm = panel_region[region] * pi / 180   # 패널 방위각 (남향에서 시계방향)(rad)

    lat = lat_region[region] * pi / 180           # 위도 (rad)
    alt = angle_change(data['altitudeMeridian']) * pi / 180    # 남중고도 (rad)
    dcl = alt + lat - pi / 2                      # 적위 = 남중고도 + 위도 - 90 (rad)

    h = 15 * (np.array(range(0, 24)) - 12) * pi / 180          # 시간 각도 (rad)



    cos_incident = sin(lat) * sin(dcl) * cos(panel_tilt) \
                    - cos(lat) * sin(dcl) * sin(panel_tilt) * cos(panel_azm) \
                    + cos(lat) * cos(dcl) * cos(h) * cos(panel_tilt) \
                    + sin(lat) * cos(dcl) * cos(h) * sin(panel_tilt) * cos(panel_azm) \
                    + cos(dcl) * sin(h) * sin(panel_tilt) * sin(panel_azm)

    # 패널에 대한 태양광 입사각 (패널이 바라보는 방향과의 각도 차이)
    incident = arccos(cos_incident) * 180 / pi     # (60분법)
    
    # 변환 : 수평으로 태양광이 비추면 0, 수직이면 90 (범위 0~90)
    incident_score = - (incident - 90)
    
    # 음수 각도 제거
    solar_incident = np.array([ang if ang >= 0 else 0 for ang in incident_score])
    
    # 효율 계산
    solar_rate = cos((90 - solar_incident) * np.pi / 180)
    solar_rate = np.where(solar_rate < 1.0e-5, 0, solar_rate)  # 너무 작은 값은 0으로 처리
    
    df = pd.DataFrame({'time': pd.date_range(date, periods=24, freq='H'),
                       'solar_incident': solar_incident,
                       'solar_rate': solar_rate})
    
    return df, dcl
    
    

In [121]:
test = pd.DataFrame()
test['date'] = pd.date_range('2020-06-01', '2020-07-14')
test['date_string'] = test['date'].dt.date.apply(lambda x: str(x))
test['date_string'] = test['date_string'].apply(lambda x: f'{x[:4]}{x[5:7]}{x[8:]}')

test['day_of_year'] = test['date'].dt.dayofyear
test['solar_dec_real'] = test['date_string'].apply(lambda date: get_incidentangle('ulsan', date, key)[1] * 180 / np.pi)
test['solar_dec_cal'] = test['day_of_year'].apply(lambda t2: 23.45 * np.sin(360 / 365 * (284 + t2) * np.pi / 180))

# 숫자가 안맞아서 2일 당김
test['solar_dec_cal'] = test['day_of_year'].apply(lambda t2: 23.45 * np.sin(360 / 365 * (286 + t2) * np.pi / 180))

# 출력
test

,date,date_string,day_of_year,solar_dec_real,solar_dec_cal
0,2020-06-01,20200601,153,22.410984,22.423707
1,2020-06-02,20200602,154,22.544318,22.538493
2,2020-06-03,20200603,155,22.660984,22.646602
3,2020-06-04,20200604,156,22.777651,22.747999
4,2020-06-05,20200605,157,22.894318,22.842656
5,2020-06-06,20200606,158,22.994318,22.930544
6,2020-06-07,20200607,159,23.094318,23.011637
7,2020-06-08,20200608,160,23.177651,23.085911
8,2020-06-09,20200609,161,23.260984,23.153344
9,2020-06-10,20200610,162,23.344318,23.213917


In [125]:
test = pd.DataFrame()
test['date'] = pd.date_range('2018-09-01', '2018-09-14')
test['date_string'] = test['date'].dt.date.apply(lambda x: str(x))
test['date_string'] = test['date_string'].apply(lambda x: f'{x[:4]}{x[5:7]}{x[8:]}')

test['day_of_year'] = test['date'].dt.dayofyear
test['solar_dec_real'] = test['date_string'].apply(lambda date: get_incidentangle('ulsan', date, key)[1] * 180 / np.pi)
test['solar_dec_cal'] = test['day_of_year'].apply(lambda t2: 23.45 * np.sin(360 / 365 * (284 + t2) * np.pi / 180))

# 숫자가 안맞아서 2일 당김
# test['solar_dec_cal'] = test['day_of_year'].apply(lambda t2: 23.45 * np.sin(360 / 365 * (286 + t2) * np.pi / 180))

# 출력
test

,date,date_string,day_of_year,solar_dec_real,solar_dec_cal
0,2018-09-01,20180901,244,8.627651,7.724629
1,2018-09-02,20180902,245,8.260984,7.342360
2,2018-09-03,20180903,246,7.894318,6.957916
3,2018-09-04,20180904,247,7.527651,6.571409
4,2018-09-05,20180905,248,7.160984,6.182956
5,2018-09-06,20180906,249,6.777651,5.792670
6,2018-09-07,20180907,250,6.410984,5.400668
7,2018-09-08,20180908,251,6.027651,5.007065
8,2018-09-09,20180909,252,5.660984,4.611979
9,2018-09-10,20180910,253,5.277651,4.215526


In [72]:
date_list = []
for a in pd.date_range('2018-03-01', '2021-02-28'):
    date_list.append(f'{a.year}{a.month:0>2}{a.day:0>2}')

date_list

['20180301',
 '20180302',
 '20180303',
 '20180304',
 '20180305',
 '20180306',
 '20180307',
 '20180308',
 '20180309',
 '20180310',
 '20180311',
 '20180312',
 '20180313',
 '20180314',
 '20180315',
 '20180316',
 '20180317',
 '20180318',
 '20180319',
 '20180320',
 '20180321',
 '20180322',
 '20180323',
 '20180324',
 '20180325',
 '20180326',
 '20180327',
 '20180328',
 '20180329',
 '20180330',
 '20180331',
 '20180401',
 '20180402',
 '20180403',
 '20180404',
 '20180405',
 '20180406',
 '20180407',
 '20180408',
 '20180409',
 '20180410',
 '20180411',
 '20180412',
 '20180413',
 '20180414',
 '20180415',
 '20180416',
 '20180417',
 '20180418',
 '20180419',
 '20180420',
 '20180421',
 '20180422',
 '20180423',
 '20180424',
 '20180425',
 '20180426',
 '20180427',
 '20180428',
 '20180429',
 '20180430',
 '20180501',
 '20180502',
 '20180503',
 '20180504',
 '20180505',
 '20180506',
 '20180507',
 '20180508',
 '20180509',
 '20180510',
 '20180511',
 '20180512',
 '20180513',
 '20180514',
 '20180515',
 '20180516',

In [73]:
df_ulsan = pd.DataFrame()

for date in date_list:
    df1 = get_incidentangle('ulsan', date, key)
    df_ulsan = pd.concat([df_ulsan, df1])

df_ulsan

,time,solar_incident,solar_rate
0,2018-03-01 00:00:00,0.0,0.0
1,2018-03-01 01:00:00,0.0,0.0
2,2018-03-01 02:00:00,0.0,0.0
3,2018-03-01 03:00:00,0.0,0.0
4,2018-03-01 04:00:00,0.0,0.0
...,...,...,...
19,2021-02-28 19:00:00,0.0,0.0
20,2021-02-28 20:00:00,0.0,0.0
21,2021-02-28 21:00:00,0.0,0.0
22,2021-02-28 22:00:00,0.0,0.0


In [74]:
df_dangjin = pd.DataFrame()

for date in date_list:
    df2 = get_incidentangle('dangjin', date, key)
    df_dangjin = pd.concat([df_dangjin, df2])
    
df_dangjin

,time,solar_incident,solar_rate
0,2018-03-01 00:00:00,0.0,0.0
1,2018-03-01 01:00:00,0.0,0.0
2,2018-03-01 02:00:00,0.0,0.0
3,2018-03-01 03:00:00,0.0,0.0
4,2018-03-01 04:00:00,0.0,0.0
...,...,...,...
19,2021-02-28 19:00:00,0.0,0.0
20,2021-02-28 20:00:00,0.0,0.0
21,2021-02-28 21:00:00,0.0,0.0
22,2021-02-28 22:00:00,0.0,0.0


In [75]:
df_ulsan.head(24)

,time,solar_incident,solar_rate
0,2018-03-01 00:00:00,0.000000,0.000000
1,2018-03-01 01:00:00,0.000000,0.000000
2,2018-03-01 02:00:00,0.000000,0.000000
3,2018-03-01 03:00:00,0.000000,0.000000
4,2018-03-01 04:00:00,0.000000,0.000000
5,2018-03-01 05:00:00,0.000000,0.000000
6,2018-03-01 06:00:00,0.000000,0.000000
7,2018-03-01 07:00:00,5.509777,0.096016
8,2018-03-01 08:00:00,19.589120,0.335273
9,2018-03-01 09:00:00,33.312979,0.549212


# feature에 solar_incident를 넣기 위한 코드 작성

In [5]:
start = pd.to_datetime('2021-02-01 00:00:00')
end = pd.to_datetime('2021-02-28 23:00:00')

# start and end time of data
time_start = start
time_end = end

# how many time tags
time_len = int((time_end - time_start) / pd.Timedelta(hours=1)) + 1

# Make base dataframe
delta = pd.to_timedelta(np.arange(time_len), unit='h')
time_base = time_start + delta

# set base
data_base = pd.DataFrame()
data_base['date'] = time_base
# data_base['time_idx'] = data_base.index
# data_base['time_idx'] = ((time_base - time_base.min()) / pd.Timedelta(hours=1)).astype('int')

# data_base['month'] = data_base['date'].dt.month_name() # string
# data_base['week_of_year'] = data_base['date'].dt.week
# data_base['day_of_month'] = data_base['date'].dt.day.apply(str)
data_base['hour_of_day'] = data_base['date'].dt.hour.apply(str)

# print log
print(f'Data starts from [ {time_start} ] to [ {time_end} ].')
print(f'Total length : {time_len}')

data_base

Data starts from [ 2021-02-01 00:00:00 ] to [ 2021-02-28 23:00:00 ].
Total length : 672


,date,hour_of_day
0,2021-02-01 00:00:00,0
1,2021-02-01 01:00:00,1
2,2021-02-01 02:00:00,2
3,2021-02-01 03:00:00,3
4,2021-02-01 04:00:00,4
...,...,...
667,2021-02-28 19:00:00,19
668,2021-02-28 20:00:00,20
669,2021-02-28 21:00:00,21
670,2021-02-28 22:00:00,22


In [21]:
data = data_base.copy()
data['dcl'] = data['date'].dt.dayofyear
data['dcl'] = data['dcl'].apply(lambda d: 23.45 * np.sin(360 / 365 * (284 + d) * np.pi / 180))
data['dcl'] = data['dcl'].apply(lambda a: a * np.pi / 180)
data['hour_angle'] = data_base['date'].dt.hour
data['hour_angle'] = data['hour_angle'].apply(lambda h: 15 * (h - 12) * np.pi / 180)
data

,date,hour_of_day,dcl,hour_angle
0,2021-02-01 00:00:00,0,-0.305721,-3.141593
1,2021-02-01 01:00:00,1,-0.305721,-2.879793
2,2021-02-01 02:00:00,2,-0.305721,-2.617994
3,2021-02-01 03:00:00,3,-0.305721,-2.356194
4,2021-02-01 04:00:00,4,-0.305721,-2.094395
...,...,...,...,...
667,2021-02-28 19:00:00,19,-0.151321,1.832596
668,2021-02-28 20:00:00,20,-0.151321,2.094395
669,2021-02-28 21:00:00,21,-0.151321,2.356194
670,2021-02-28 22:00:00,22,-0.151321,2.617994


In [22]:
region = 'ulsan'

# 패널 설치 정보
panel_region = {'ulsan': 20, 'dangjin': 30}
lat_region = {'ulsan': 35.4776509, 'dangjin': 37.0507527}

sin = np.sin
cos = np.cos
arccos = np.arccos
pi = np.pi

panel_tilt = panel_region[region] * pi / 180  # 패널 설치각(rad)
panel_azm = panel_region[region] * pi / 180   # 패널 방위각 (남향에서 시계방향)(rad)

lat = lat_region[region] * pi / 180           # 위도 (rad)

data['cos_incident'] = cos(data['hour_angle']) * sin(lat) * cos(data['dcl'])

data['cos_incident'] = sin(lat) * sin(data['dcl']) * cos(panel_tilt) \
                - cos(lat) * sin(data['dcl']) * sin(panel_tilt) * cos(panel_azm) \
                + cos(lat) * cos(data['dcl']) * cos(data['hour_angle']) * cos(panel_tilt) \
                + sin(lat) * cos(data['dcl']) * cos(data['hour_angle']) * sin(panel_tilt) * cos(panel_azm) \
                + cos(data['dcl']) * sin(data['hour_angle']) * sin(panel_tilt) * sin(panel_azm)
data['solar_incident'] = arccos(data['cos_incident']) * 180 / pi
data['solar_incident'] = - (data['solar_incident'] - 90)
data['solar_incident'] = data['solar_incident'].apply(lambda x: x if x>=0 else 0)

data.head(24)

,date,hour_of_day,dcl,hour_angle,cos_incident,solar_incident
0,2021-02-01 00:00:00,0,-0.305721,-3.141593,-0.993006,0.000000
1,2021-02-01 01:00:00,1,-0.305721,-2.879793,-0.990952,0.000000
2,2021-02-01 02:00:00,2,-0.305721,-2.617994,-0.927184,0.000000
3,2021-02-01 03:00:00,3,-0.305721,-2.356194,-0.806048,0.000000
4,2021-02-01 04:00:00,4,-0.305721,-2.094395,-0.635799,0.000000
5,2021-02-01 05:00:00,5,-0.305721,-1.832596,-0.428040,0.000000
6,2021-02-01 06:00:00,6,-0.305721,-1.570796,-0.196929,0.000000
7,2021-02-01 07:00:00,7,-0.305721,-1.308997,0.041784,2.394726
8,2021-02-01 08:00:00,8,-0.305721,-1.047198,0.271831,15.773258
9,2021-02-01 09:00:00,9,-0.305721,-0.785398,0.477535,28.524563


In [ ]:
region = 'ulsan'

# 패널 설치 정보
panel_region = {'ulsan': 20, 'dangjin': 30}
lat_region = {'ulsan': 35.4776509, 'dangjin': 37.0507527}

sin = np.sin
cos = np.cos
arccos = np.arccos
pi = np.pi

panel_tilt = panel_region[region] * pi / 180  # 패널 설치각(rad)
panel_azm = panel_region[region] * pi / 180   # 패널 방위각 (남향에서 시계방향)(rad)

lat = lat_region[region] * pi / 180           # 위도 (rad)
alt = angle_change(data['altitudeMeridian']) * pi / 180    # 남중고도 (rad)
dcl = alt + lat - pi / 2                      # 적위 = 남중고도 + 위도 - 90 (rad)

h = 15 * (np.array(range(0, 24)) - 12) * pi / 180          # 시간 각도 (rad)



cos_incident = sin(lat) * sin(dcl) * cos(panel_tilt) \
                - cos(lat) * sin(dcl) * sin(panel_tilt) * cos(panel_azm) \
                + cos(lat) * cos(dcl) * cos(h) * cos(panel_tilt) \
                + sin(lat) * cos(dcl) * cos(h) * sin(panel_tilt) * cos(panel_azm) \
                + cos(dcl) * sin(h) * sin(panel_tilt) * sin(panel_azm)

# 패널에 대한 태양광 입사각 (패널이 바라보는 방향과의 각도 차이)
incident = arccos(cos_incident) * 180 / pi     # (60분법)

# 변환 : 수평으로 태양광이 비추면 0, 수직이면 90 (범위 0~90)
incident_score = - (incident - 90)

# 음수 각도 제거
solar_incident = np.array([ang if ang >= 0 else 0 for ang in incident_score])

# 효율 계산
solar_rate = cos((90 - solar_incident) * np.pi / 180)
solar_rate = np.where(solar_rate < 1.0e-5, 0, solar_rate)  # 너무 작은 값은 0으로 처리

df = pd.DataFrame({'time': pd.date_range(date, periods=24, freq='H'),
                   'solar_incident': solar_incident,
                   'solar_rate': solar_rate})

return df, dcl